<a href="https://colab.research.google.com/github/khuramgill/Face-Embaddings/blob/main/Image_with_URL's.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install open_clip_torch pinecone-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 695.6 kB/s eta 0:00:00


In [2]:
!pip install --upgrade pinecone-client

In [3]:
import open_clip
from PIL import Image
import torch
import pinecone


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


Using device: cpu


In [5]:
from pinecone import Pinecone

pc = Pinecone(api_key="pcsk_2agnRW_PJGcuYNduPh4LXPttu1bnT31UninNBGbyZqKDUiCDAxdY7asgDJkLHmR3KCDJzP")
index = pc.Index("t")

# Connect


In [6]:
model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(
    'hf-hub:laion/CLIP-ViT-B-32-laion2B-s34B-b79K'
)
tokenizer = open_clip.get_tokenizer('hf-hub:laion/CLIP-ViT-B-32-laion2B-s34B-b79K')

model = model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

open_clip_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

In [8]:
# Load image titles and URLs from text file
image_data = []
with open("/content/image_links.txt", "r") as file:
    for line in file:
        title, url = line.strip().split(": ")
        image_data.append((title, url))

print("Image data loaded successfully.")
print(f"First 5 entries: {image_data[:5]}")


Image data loaded successfully.
First 5 entries: [('44.jpg', 'https://drive.google.com/uc?id=1nIrw9H-NDXhjO_8K0_Myzjp8tT_fONxW'), ('48.jpg', 'https://drive.google.com/uc?id=1XMjTZ4NtKMkHXgJwhbBlpmbGL0QROTDA'), ('50.jpg', 'https://drive.google.com/uc?id=1J6ALJHD2MSc5tI5Sod7r3AdlUnCsWQxu'), ('47.jpg', 'https://drive.google.com/uc?id=1peTtrAZIFAX3g46erMA6aZoeSSXzVwrO'), ('6.jpg', 'https://drive.google.com/uc?id=1uZe1BWLCuBLC5kQ2yq_VC5UhtpEm1vuN')]


In [9]:
from PIL import Image
import requests
from io import BytesIO

def get_image_embedding(image_url):
    try:
        response = requests.get(image_url)
        image = Image.open(BytesIO(response.content)).convert("RGB")

        embedding = model.encode(image, convert_to_tensor=True).detach().cpu().numpy()
        return embedding
    except Exception as e:
        print(f"Error processing image at {image_url}: {e}")
        return None

print("Image embedding function defined.")


Image embedding function defined.


In [10]:
for title, url in image_data:
    embedding = get_image_embedding(url)
    if embedding is not None:
        index.upsert([(title, embedding, {"url": url})])  # Save embedding and metadata to Pinecone

print("Embeddings and URLs uploaded to Pinecone.")


Error processing image at https://drive.google.com/uc?id=1nIrw9H-NDXhjO_8K0_Myzjp8tT_fONxW: 'CLIP' object has no attribute 'encode'
Error processing image at https://drive.google.com/uc?id=1XMjTZ4NtKMkHXgJwhbBlpmbGL0QROTDA: 'CLIP' object has no attribute 'encode'
Error processing image at https://drive.google.com/uc?id=1J6ALJHD2MSc5tI5Sod7r3AdlUnCsWQxu: 'CLIP' object has no attribute 'encode'
Error processing image at https://drive.google.com/uc?id=1peTtrAZIFAX3g46erMA6aZoeSSXzVwrO: 'CLIP' object has no attribute 'encode'
Error processing image at https://drive.google.com/uc?id=1uZe1BWLCuBLC5kQ2yq_VC5UhtpEm1vuN: 'CLIP' object has no attribute 'encode'
Error processing image at https://drive.google.com/uc?id=1J4Cq5o8OTPr3lacf3GG4vX4XuYsBK1ns: 'CLIP' object has no attribute 'encode'
Error processing image at https://drive.google.com/uc?id=1UN_LOwKydi_2dhlvv4nR00iTjr3PEGsu: 'CLIP' object has no attribute 'encode'
Error processing image at https://drive.google.com/uc?id=1TSkrlof8XXjwhwFyj_

In [11]:
def search_image(query_image_url, top_k=5):
    query_embedding = get_image_embedding(query_image_url)
    if query_embedding is None:
        print("Failed to generate embedding for query image.")
        return

    results = index.query(query_embedding, top_k=top_k, include_metadata=True)
    print("Top matches:")
    for match in results['matches']:
        print(f"Title: {match['id']}, URL: {match['metadata']['url']}, Score: {match['score']}")

print("Search function defined.")


Search function defined.


In [12]:
# Example: Searching with a query image
query_image_url = "https://drive.usercontent.google.com/download?id=1J6ALJHD2MSc5tI5Sod7r3AdlUnCsWQxu&authuser=0"  # Replace with an actual URL
search_image(query_image_url)


Error processing image at https://drive.usercontent.google.com/download?id=1J6ALJHD2MSc5tI5Sod7r3AdlUnCsWQxu&authuser=0: 'CLIP' object has no attribute 'encode'
Failed to generate embedding for query image.
